## LinkedIn Jobs Scrapper in Python

In [263]:
import requests, time, random
import pandas as pd
import os
from bs4 import BeautifulSoup
from selenium import webdriver
from tqdm import tqdm_notebook
import pickle

In [264]:
# Подключаемся и авторизуемся

browser = webdriver.Chrome('driver/chromedriver')
browser.get('https://www.linkedin.com/uas/login')

username = 'alev4enko@mail.ru'
password = 'Alex1324!'

# username = 'supermariolanza@gmail.com'
# password = 'supermariolanza'

elementID = browser.find_element_by_id('username')
elementID.send_keys(username)

elementID = browser.find_element_by_id('password')
elementID.send_keys(password)

elementID.submit()

# Поиск вакансий по запросу

In [266]:
# Talent Acquisition manager / t&d manager /
# Hr director / HRBP

link = 'https://www.linkedin.com/jobs/search/?keywords=Talent%20Acquisition%20manager&location=United%20States'
link1 = 'https://www.linkedin.com/jobs/search/?geoId=103644278&keywords=hr%20manager&location=United%20States'
link2 = 'https://www.linkedin.com/jobs/search/?geoId=103644278&keywords=Hr%20director%20&location=United%20States'
link3 = 'https://www.linkedin.com/jobs/search/?geoId=103644278&keywords=hr%20HRBP&location=United%20States'
link4 = 'https://www.linkedin.com/jobs/search/?keywords=Learning%20and%20Development%20manager'

In [262]:
alljoblinks = []

In [287]:
browser.get(link4)
scroller()

In [290]:
src = browser.page_source
soup = BeautifulSoup(src, 'lxml')

In [291]:
joblink_ids = get_link_ids(soup)
joblinks = get_joblinks(joblink_ids)
alljoblinks.extend(joblinks)

In [292]:
save_model('alljoblinks','',alljoblinks)

In [293]:
alljoblinks = list(set(alljoblinks))

In [294]:
len(alljoblinks)

225

In [296]:
jobskills = []
badlinks = []



# jobskills = load_model('', 'jobskills.pickle')
jj = [x[0] for x in jobskills]
links_to_skan = [x for x in alljoblinks if x[0] not in jj]
links_to_skan = [x for x in links_to_skan if x not in badlinks]

In [297]:
len(links_to_skan)

225

In [298]:
for jlink in tqdm_notebook(links_to_skan):
    browser.get(jlink)
#     scroller()
    time.sleep(1)
    src = browser.page_source
    soup = BeautifulSoup(src, 'lxml')
    try:
        jtitle = soup.find('h1', {'class':'jobs-top-card__job-title t-24'}).text
        jskills = get_job_skills(soup)
        jobskills.append([jlink,jtitle,jskills])
        save_model('jobskills','',jobskills)
    except AttributeError:
        print(jlink)
        badlinks.append(jlink)
        
    

https://www.linkedin.com/jobs/view/1709963455/?eBP=NotAvailableFromVoyagerAPI&recommendedFlavor=HIDDEN_GEM&refId=ae6eff83-3e9e-4249-a383-e13a400ca113&trk=d_flagship3_search_srp_jobs



In [ ]:
# jskills

In [251]:
pd.DataFrame(jobskills)

,0,1,2
0,https://www.linkedin.com/jobs/view/1747011385/...,Director of HR,"[Human Resources (HR), Senior Professional in ..."
1,https://www.linkedin.com/jobs/view/1702361331/...,Talent Acquisition Manager,"[Sourcing, Data Analysis, Communication, Full-..."
2,https://www.linkedin.com/jobs/view/1752758686/...,Recruiter- Talent Acquisition Manager,"[Onboarding, Customer Support, HITRUST, Job Po..."
3,https://www.linkedin.com/jobs/view/1569867889/...,HR Director,"[Human Resources (HR), Talent Management, Labo..."
4,https://www.linkedin.com/jobs/view/1756636618/...,HR Manager,"[Process Improvement, Labor Relations, SAP Suc..."
5,https://www.linkedin.com/jobs/view/1578220186/...,Human Resources Manager,"[Human Resources (HR), Reporting & Analysis, T..."
6,https://www.linkedin.com/jobs/view/1747011385/...,Director of HR,"[Human Resources (HR), Senior Professional in ..."
7,https://www.linkedin.com/jobs/view/1702361331/...,Talent Acquisition Manager,"[Sourcing, Data Analysis, Communication, Full-..."
8,https://www.linkedin.com/jobs/view/1752758686/...,Recruiter- Talent Acquisition Manager,"[Onboarding, Customer Support, HITRUST, Job Po..."
9,https://www.linkedin.com/jobs/view/1569867889/...,HR Director,"[Human Resources (HR), Talent Management, Labo..."


In [250]:
jobskills = [x for x in jobskills if x[-1] is not None]

In [143]:
save_model('Talent_Acquisition','',alljoblinks)

# Functions

In [69]:
def get_link_ids(soup):
    cls_a = 'job-card-search__link-wrapper js-focusable disabled ember-view'
    links = soup.find_all('a', {'class':cls_a})
    link_ids = []
    if len(links) > 0:
        for el in links:
            if el.attrs.get('id') is not None:
                link_ids.append(el.attrs['id'])
        return link_ids
    else:
        return None

In [267]:
def get_joblinks(joblink_ids):
    homelink = 'https://www.linkedin.com'
    links = []
    for lid in joblink_ids:
        rlink = soup.find('a', {'id':lid}).attrs['href']
        fulllink = homelink + rlink
        links.append(fulllink)
    return links

In [268]:
def get_job_skills(soup):
    cls = 'jobs-ppc-criteria__list jobs-ppc-criteria__list--skills jobs-ppc-criteria__list--is-last js-criteria-skills list-style-none'
    cls_sp = 'jobs-ppc-criteria__value t-14 t-black t-normal ml2 block'
    ul = soup.find('ul', {'class':cls})
    if ul is not None:
        skills = [el.find('span',{'class':cls_sp}).text.strip() for el in ul.find_all('li')]
        return skills
    else :
        return None

In [270]:
def get_hidden_skills(soup):
    """ 
    Достает скрытые навыми
    """
    
    div_cls = "pv-skill-categories-section__expanded"
    a_cls = "display-block full-width ember-view"
    div_new = soup.find('div', {'class':div_cls})
    lis = div_new.find_all('a', {'class':a_cls} )
    hidden_skills = [el.text.strip() for el in lis]
    
    return hidden_skills

def profile_links_saver(profile_pages,kw,region):
    
    """ Сохраняет ссылки на профили в csv файл """
    
    list_of_profs = []
    
    for prof in profile_pages:
        list_of_profs.append([kw,region,prof])

    prof_df = pd.DataFrame(
                 list_of_profs,
                 columns=['keyword','region', 'url']
                 )
    fpath = f'p_links_{kw}_{region}.csv'
    prof_df.to_csv(fpath,index=False)
    return prof_df
    

def searcher(kw, region, rangestart=1, rangestop=100):
    
    """ Собирает ссылки на профили по результатам поиска """
    c = 0 
    profile_pages = []
    for i in tqdm_notebook(range(rangestart,rangestop)):
        searchlink = get_search_link(kw, region, i)
        browser.get(searchlink)
        scroller()
        src = browser.page_source
        soup = BeautifulSoup(src, 'lxml')
        try:
            profile_pages.extend(get_profile_urls(soup))
        except AttributeError:
            print('AttributeError', i)
            c += 1
            if c > 3:
                break
    
    profile_pages = list(set(profile_pages))
    return profile_pages

def get_profile_urls(soup):
    
    """ Достает ссылки на профили из супа результата поиска """
    
    profile_links = soup.find('ul',
                  {'class':"search-results__list list-style-none"}
                         ).find_all('a')
    urls = []
    for el in profile_links:
        upart = el.attrs['href']
        url = 'https://www.linkedin.com'
        url += upart
        urls.append(url)
    #     print(url)
    urls = list(set(urls))
    return urls
    

def get_search_link(kw, region,page_num=None):
    
    """ Формирует ссылку для поиска """
    
    link = 'https://www.linkedin.com/search/results/people/'
    link += f'?facetGeoRegion=%5B%22{region}%3A0%22%5D&'
    link += f'keywords={kw}'
    link += '&origin=FACETED_SEARCH'
    if page_num not in [None, 0,1]:
        link += f'&page={page_num}'
    return link
    

def get_name(soup):
    
    """ Достаем имя пользователя """
    
    name_div = soup.find('div', {'class': 'flex-1 mr5'})
    name_div
    name_loc = name_div.find_all('ul')
    name = name_loc[0].find('li').get_text().strip()
    return name

def get_jobtitle(soup):
    
    """ Достаем название позиции """
    
    exp_section = soup.find('section', {'id': 'experience-section'})
    if exp_section is None:
        raise AttributeError
    else:
        exp_section = exp_section.find('ul')
        li_tags = exp_section.find('div')
        a_tags = li_tags.find('a')
        job_title = a_tags.find('h3').get_text().strip()
        return job_title

def get_skills(soup):
    
    """ Достаем навыки пользователя """ 
    
    spclass = "pv-skill-category-entity__name-text t-16 t-black t-bold"
    skill_cls = 'pv-profile-section pv-skill-categories-section artdeco-container-card ember-view'
    sk = soup.find('section', {'class': skill_cls}).findAll("span", {"class": spclass})
    skills = []
    for el in sk:
        skills.append(el.text.strip())
    return skills

def user_info(soup):
    """ Возвращает список данных о пользователе"""
    user_info = []
    user_info.append(get_name(soup))
    user_info.append(get_jobtitle(soup))
    open_skills = get_skills(soup)
    hidden_skills = get_hidden_skills(soup)
    skills_dict = {'open skills':open_skills, 'hidden skills' :hidden_skills}
    user_info.append(skills_dict)
    
    return user_info

def scroller():
    
    """ Скроллим экран """
    
    SCROLL_PAUSE_TIME = 1
    # Get scroll height
    last_height = browser.execute_script("return document.body.scrollHeight")
    for st in range(0,last_height,400):
        time.sleep(0.2)
        browser.execute_script(f"window.scrollTo(0,{st});")
    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

def click_see_more():
    src = browser.page_source
    soup = BeautifulSoup(src, 'lxml')
    spclass = "pv-skill-category-entity__name-text t-16 t-black t-bold"
    skill_cls = 'pv-profile-section pv-skill-categories-section artdeco-container-card ember-view'
    div = soup.find('section', {'class': skill_cls}).find('div', 
                                               {'class':"ember-view"}
                                               )
    try:
        if div.attrs.get('id') is not None:
            div_id = div.attrs.get('id')
            browser.find_element_by_id(div_id).click()
    except AttributeError:
        return None
   
    

def find_see_more(soup):
    spclass = "pv-skill-category-entity__name-text t-16 t-black t-bold"
    skill_cls = 'pv-profile-section pv-skill-categories-section artdeco-container-card ember-view'
    div = soup.find('section', {'class': skill_cls}).find('div', 
                                               {'class':"ember-view"}
                                               )
    try:
        if div.attrs.get('id') is not None:
            div_id = div.attrs.get('id')
            element = browser.find_element_by_id(div_id)
            return element.location['y']
    except AttributeError:
        return None
        
        

def scroller_back():
    
    """ Скроллим экран """
    
    SCROLL_PAUSE_TIME = 1
    # Get scroll height
    
    last_height = find_see_more(soup)
    if last_height is None:
        return None
    else:
        for st in range(0, last_height - 400 ,200):
            time.sleep(0.2)
            browser.execute_script(f"window.scrollTo(0,{st});")
        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

def save_model(model_name, folder_name, model):
    '''
    Сохраняем объекты в pickle
    '''

    fname = model_name + '.pickle'
    fpath = os.path.join(folder_name, fname)
    with open(fpath, 'wb') as f:
        pickle.dump(model,f)
#     print(fname + '...done!')

def load_model(model_folder, filename):
    '''
    Загружаем объекты из pickle
    '''

    with open(os.path.join(model_folder,filename), 'rb') as f:
        loaded_model = pickle.load(f)
    return loaded_model

In [703]:
src = browser.page_source
soup = BeautifulSoup(src, 'lxml')
find_see_more(soup)

3369

In [700]:
browser.execute_script("return document.body.scrollHeight")

4668

In [628]:
div = soup.find('section', {'class': skill_cls}).find('div', 
                                               {'class':"ember-view"}
                                               )

In [635]:
if div.attrs.get('id') is not None:
    div_id = div.attrs.get('id')
    print(div.attrs.get('id'))

ember593


In [624]:
spclass = "pv-skill-category-entity__name-text t-16 t-black t-bold"
skill_cls = 'pv-profile-section pv-skill-categories-section artdeco-container-card ember-view'
sk = soup.find('section', {'class': skill_cls}).findAll("span", {"class": spclass})


In [639]:
src = browser.page_source
soup = BeautifulSoup(src, 'lxml')

In [708]:
# soup

In [655]:
div_new = soup.find('div', {'class':"pv-skill-categories-section__expanded" })
h3s = div_new.find_all('h3')





In [654]:
div_new.next_element

'\n'

In [656]:
for el in h3s:
    print(el.text.strip())

Industry Knowledge
Tools & Technologies
Interpersonal Skills
Languages
Other Skills


In [666]:
lis = div_new.find_all('a', {'class':"display-block full-width ember-view"})

In [667]:
for el in lis:
    print(el.text.strip())

Social Networking
Social Media
PowerPoint
Microsoft Office
Microsoft Excel
Microsoft PowerPoint
Time Management
Teamwork


In [670]:
get_hidden_skills(soup)

['Social Networking',
 'Social Media',
 'PowerPoint',
 'Microsoft Office',
 'Microsoft Excel',
 'Microsoft PowerPoint',
 'Time Management',
 'Teamwork']